In [1]:
import string
import numpy
import requests
import urllib.request
import json
from pprint import pprint
from bs4 import BeautifulSoup
import re
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import brown
import os

In [ ]:
PATH_TO_PROJECT = ""
PATH_TO_RAW_FILES = "data/raw/"
PATH_TO_PROCESSED_FILES = "data/processed/"

In [ ]:
# gets raw files from raw and extracts the artist's names
raw_files = os.listdir(PATH_TO_RAW_FILES)
artists = []
for filename in raw_files:
    artists.append(filename.replace(".json",""))

In [ ]:
data = {}
for artist in artists:
    with open(PATH_TO_RAW_FILES + artist + ".json", encoding = 'utf-8') as data_file:    
        data[artist] = json.load(data_file)

In [ ]:
# check success rate per artist
count_song = 0
count_fail = 0
for artist in data:
    count_song_artist = 0
    count_fail_artist = 0
    for album in data[artist]["albums"]:
        for song in album["songs"]:
            count_song += 1
            count_song_artist += 1
            if (len(song["lyrics"]) == 0):
                count_fail += 1
                count_fail_artist += 1
    success_rate = 0
    if (count_song_artist > 0):
        success_rate = (count_song_artist - count_fail_artist) / count_song_artist
    data[artist]["success rate"] = success_rate
    data[artist]["total songs"] = count_song_artist
    data[artist]["songs failed"] = count_fail_artist

In [ ]:
len(data)

In [ ]:
old_artists = []
for artist in data:
    old_artists = artists
for old_artist in old_artists:
    if (old_artist in data and data[old_artist]["total songs"] - data[old_artist]["songs failed"] < 10):
        del data[old_artist]

# Cleaning and Counting Lyrics

In [ ]:
def clean_words(wordlist):
    stop = set(stopwords.words('english'))
    stop = stop.union(set(('the', 'one', 'i')))
    wordlist = [s.lower() for s in wordlist]
    wordlist = [s for s in wordlist if s not in stop]
    wordlist = [''.join(c for c in s if c not in string.punctuation) for s in wordlist]
    wordlist = [s for s in wordlist if s]    
    return wordlist

In [ ]:
def stem(wordlist): #Use Dict stemmer
    newlist = []
    for word in wordlist:
        newlist.append(SnowballStemmer("english").stem(word))
    return newlist    

In [ ]:
def fill_frequencies(wordlists, frequencies, V):
    x = 0
    for i in range(len(wordlists)):
        for word in wordlists[i]:
            if word in V:
                frequencies[i][V.index(word)] += 1
    return frequencies

In [ ]:
def remove_brackets(mystring):
    while('[' in mystring):
        x = mystring.index('[')
        y = mystring.index(']')
        mystring = mystring[:x] + mystring[y+1:]
        print(mystring)
    return mystring

In [ ]:
def json_to_list(data):
    wordlist = []
    for album in data["albums"]:
        for song in album["songs"]:
            mystring = remove_brackets(song["lyrics"])
            wordlist += list(mystring.split())
    
    return wordlist

In [ ]:
# have artists in correct order
artists = []
for artist in data:
    artists.append(artist)

In [ ]:
wordlists = []
for artist in artists:
    # lyrics to list count
    wordlist = json_to_list(data[artist])
    
    # clean and stem
    wordlist = clean_words(wordlist)
    wordlist = stem(wordlist)
    wordlist = clean_words(wordlist)
    
    # add to artist and remove albums
    #     data[artist]["word list"] = wordlist #DON'T NEED FOR NOW, might want later
    del data[artist]["albums"]
    
    wordlists.append(json_to_list(data[artist]))

In [ ]:
V = [ite for ite, it in Counter(wordlists).most_common(NUMBER_OF_WORDS)]
# print(V[:50], "\n")

In [ ]:
frequencies = numpy.zeros((len(wordlists), num_words)) #tall(y), long(x)
frequencies = fill_frequencies(wordlists, frequencies, V)
print(frequencies)

In [ ]:
# add back to correct artists
for i in range(len(artists)):
    data[artists[i]]["frequencies"] = frequencies[i] 

In [ ]:
# TODO: output results to processed folder with CSV output for artists, frequencies